# DocIntel Demo Notebook

Interactive demonstration of the DocIntel multi-agent document intelligence system.

## Prerequisites

1. Docker services running: `docker compose up -d`
2. Test documents seeded: `docker exec docintel-rag node /app/scripts/seed-test-documents.js`
3. Services accessible:
   - RAG Backend: http://localhost:3000
   - Agent System: http://localhost:8000

In [ ]:
# Install required packages
!pip install requests pandas matplotlib ipywidgets -q

In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
RAG_BASE_URL = "http://localhost:3000"
AGENT_BASE_URL = "http://localhost:8000"

print("✅ Imports successful")

## 1. Health Checks

Verify all services are running.

In [ ]:
def check_health(service_name, url):
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: Healthy")
            return True
        else:
            print(f"❌ {service_name}: Unhealthy (status {response.status_code})")
            return False
    except Exception as e:
        print(f"❌ {service_name}: Unreachable - {e}")
        return False

print("Checking services...\n")
check_health("RAG Backend", f"{RAG_BASE_URL}/api/health")
check_health("Agent System", f"{AGENT_BASE_URL}/health")
check_health("Prometheus", "http://localhost:9090/-/healthy")
check_health("Grafana", "http://localhost:3001/api/health")

## 2. RAG Backend - Search Modes

Test all three search modes: Lexical, Semantic, and Hybrid.

In [ ]:
def search_documents(query, mode="hybrid"):
    """Search documents using RAG backend."""
    url = f"{RAG_BASE_URL}/api/unified-search"
    payload = {"query": query, "mode": mode}
    
    response = requests.post(url, json=payload, stream=True)
    
    sources = []
    answer = ""
    
    for line in response.iter_lines():
        if line:
            line_str = line.decode('utf-8')
            if line_str.startswith('data: '):
                data = json.loads(line_str[6:])
                if data.get('type') == 'sources':
                    sources = data.get('sources', [])
                elif data.get('type') == 'text':
                    answer += data.get('content', '')
    
    return {"answer": answer, "sources": sources}

# Test query
query = "What was the Q3 2024 portfolio performance?"
print(f"Query: {query}\n")

# Lexical search
print("\n=== Lexical Search (BM25) ===")
result = search_documents(query, mode="lexical")
print(f"Sources: {len(result['sources'])}")
display(Markdown(result['answer'][:500] + "..."))

In [ ]:
# Hybrid search (best results)
print("\n=== Hybrid Search (Lexical + Semantic) ===")
result = search_documents(query, mode="hybrid")
print(f"\nSources found: {len(result['sources'])}")
for i, source in enumerate(result['sources'], 1):
    print(f"{i}. {source['fileName']} (score: {source['score']:.2f})")

print("\nAnswer:")
display(Markdown(result['answer']))

## 3. Agent System - Basic Query

Test the multi-agent orchestration system.

In [ ]:
def agent_query(query, session_id=None, pattern="sequential"):
    """Query the agent system."""
    url = f"{AGENT_BASE_URL}/query"
    payload = {
        "query": query,
        "session_id": session_id,
        "execution_pattern": pattern
    }
    
    start = time.time()
    response = requests.post(url, json=payload)
    duration = time.time() - start
    
    if response.status_code == 200:
        result = response.json()
        result['_duration'] = duration
        return result
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Test query
print("Testing agent system (may take 10-15 seconds)...\n")
result = agent_query("What were the Q3 2024 results?", session_id="demo-001")

if result:
    print(f"✅ Query completed in {result['_duration']:.2f} seconds")
    print(f"Workflow ID: {result['workflow_id']}")
    print(f"Tasks executed: {result['total_tasks']}")
    print(f"Execution pattern: {result['execution_pattern']}")
    print("\nFinal Answer:")
    display(Markdown(result['result']['final_answer']))

## 4. Session Management

Demonstrate conversation history and session persistence.

In [ ]:
# Create session
def create_session(user_id=None):
    url = f"{AGENT_BASE_URL}/sessions"
    response = requests.post(url, json={"user_id": user_id})
    return response.json()

# Get session messages
def get_messages(session_id):
    url = f"{AGENT_BASE_URL}/sessions/{session_id}/messages"
    response = requests.get(url)
    return response.json()

# Create new session
session = create_session(user_id="demo-user")
print(f"Created session: {session['session_id']}\n")

# Ask multiple questions
questions = [
    "What was the Q3 2024 IRR?",
    "How does that compare to Q2?"
]

for q in questions:
    print(f"Q: {q}")
    # Note: Will hit Gemini quota - wait 60s between queries in production
    time.sleep(60)  # Wait for Gemini quota
    result = agent_query(q, session_id=session['session_id'])
    if result:
        print(f"A: {result['result']['final_answer'][:200]}...\n")

# Get conversation history
messages = get_messages(session['session_id'])
print(f"\nConversation history ({len(messages['messages'])} messages):")
for msg in messages['messages']:
    print(f"[{msg['role'].upper()}]: {msg['content'][:100]}...")

## 5. Execution Patterns Comparison

Compare Sequential vs Parallel execution.

In [ ]:
complex_query = "Compare performance metrics across all portfolio companies"

print("Testing execution patterns...\n")

# Sequential
print("1. Sequential execution:")
result_seq = agent_query(complex_query, pattern="sequential")
if result_seq:
    print(f"   Duration: {result_seq['duration_seconds']:.2f}s")
    print(f"   Tasks: {result_seq['total_tasks']}")

time.sleep(60)  # Wait for Gemini quota

# Parallel
print("\n2. Parallel execution:")
result_par = agent_query(complex_query, pattern="parallel")
if result_par:
    print(f"   Duration: {result_par['duration_seconds']:.2f}s")
    print(f"   Tasks: {result_par['total_tasks']}")
    
    speedup = result_seq['duration_seconds'] / result_par['duration_seconds']
    print(f"\n   Speedup: {speedup:.2f}x faster")

## 6. Memory Bank

Store and retrieve long-term memories.

In [ ]:
# Store memory
def store_memory(content, memory_type="fact", importance=0.8, tags=None):
    url = f"{AGENT_BASE_URL}/memory"
    payload = {
        "content": content,
        "memory_type": memory_type,
        "user_id": "demo-user",
        "importance": importance,
        "tags": tags or []
    }
    response = requests.post(url, json=payload)
    return response.json()

# Retrieve memories
def get_memories(user_id="demo-user", min_importance=0.5):
    url = f"{AGENT_BASE_URL}/memory"
    params = {"user_id": user_id, "min_importance": min_importance}
    response = requests.get(url, params=params)
    return response.json()

# Store some facts
memories_to_store = [
    {"content": "Q3 2024 IRR was 15%", "type": "fact", "importance": 0.9, "tags": ["Q3", "IRR"]},
    {"content": "User prefers detailed financial analysis", "type": "preference", "importance": 0.8, "tags": ["preferences"]},
    {"content": "Portfolio shows strong fintech sector performance", "type": "insight", "importance": 0.7, "tags": ["trends"]}
]

print("Storing memories...\n")
for mem in memories_to_store:
    result = store_memory(mem["content"], mem["type"], mem["importance"], mem["tags"])
    print(f"✅ Stored: {mem['content'][:50]}...")

# Retrieve
print("\nRetrieving memories (importance >= 0.7):\n")
memories = get_memories(min_importance=0.7)
for mem in memories['memories']:
    print(f"- [{mem['memory_type'].upper()}] {mem['content']}")
    print(f"  Importance: {mem['importance']}, Tags: {mem['tags']}\n")

## 7. Checkpointing (Human-in-Loop)

Save and restore session state.

In [ ]:
# Create checkpoint
def create_checkpoint(session_id):
    url = f"{AGENT_BASE_URL}/sessions/{session_id}/checkpoint"
    response = requests.post(url)
    return response.json()

# Restore checkpoint
def restore_checkpoint(checkpoint_id):
    url = f"{AGENT_BASE_URL}/checkpoints/{checkpoint_id}/restore"
    response = requests.post(url)
    return response.json()

# Create a session with some activity
session = create_session(user_id="checkpoint-demo")
print(f"Created session: {session['session_id']}\n")

# Ask a question
time.sleep(60)  # Gemini quota
agent_query("What was Q3 performance?", session_id=session['session_id'])

# Create checkpoint
checkpoint = create_checkpoint(session['session_id'])
print(f"\n✅ Checkpoint created: {checkpoint['checkpoint_id']}")
print("   You can now restore this state later to continue the conversation.")

# Later: Restore
# restored = restore_checkpoint(checkpoint['checkpoint_id'])
# print(f"Restored session: {restored['session_id']}")

## 8. Observability - Metrics

Query Prometheus for system metrics.

In [ ]:
def query_prometheus(query):
    url = "http://localhost:9090/api/v1/query"
    response = requests.get(url, params={"query": query})
    return response.json()

# Get metrics
metrics = [
    ("Active Sessions", "agent_active_sessions"),
    ("Total Workflows", "agent_workflow_total"),
    ("LLM Tokens Used", "agent_llm_tokens_total")
]

print("System Metrics:\n")
for label, query in metrics:
    result = query_prometheus(query)
    if result['status'] == 'success' and result['data']['result']:
        value = result['data']['result'][0]['value'][1]
        print(f"{label}: {value}")
    else:
        print(f"{label}: No data")

## Summary

This notebook demonstrated:

1. ✅ **Health Checks** - Verify all services
2. ✅ **Search Modes** - Lexical, Semantic, Hybrid
3. ✅ **Agent Queries** - Multi-agent orchestration
4. ✅ **Sessions** - Conversation history
5. ✅ **Execution Patterns** - Sequential vs Parallel
6. ✅ **Memory Bank** - Long-term storage
7. ✅ **Checkpointing** - Save/restore state
8. ✅ **Observability** - Metrics and monitoring

**Next Steps**:
- See `evaluation.ipynb` for Kaggle competition criteria demonstrations
- Check Grafana dashboards: http://localhost:3001
- View traces in Jaeger: http://localhost:16686